In [17]:
import pandas as pd
import re 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [19]:
df = pd.read_csv('data/cleaned_tweets.csv')
df

,unnamed: 0,post_text,clean_text,created_at
0,0,It's just over 2 years since I was diagnosed w...,it's just over 2 years since i was diagnosed w...,NaN
1,1,"It's Sunday, I need a break, so I'm planning t...","it's sunday, i need a break, so i'm planning t...",NaN
2,2,Awake but tired. I need to sleep but my brain ...,awake but tired. i need to sleep but my brain ...,NaN
3,3,RT @SewHQ: #Retro bears make perfect gifts and...,rt : #retro bears make perfect gifts and are g...,NaN
4,4,It’s hard to say whether packing lists are mak...,it’s hard to say whether packing lists are mak...,NaN
...,...,...,...,...
20095,20095,@AbnRgr504 @LangmanVince That makes sense...sa...,that makes sense...sad...they should be the on...,2025-06-10 19:44:18+00:00
20096,20096,@stacy_muur @opensea @monad_xyz @aztecnetwork ...,alpha will fucked us,2025-06-10 19:44:18+00:00
20097,20097,@shellshockkk That’s some fucked up shit right...,that’s some fucked up shit right there. how mu...,2025-06-10 19:44:18+00:00
20098,20098,damn ik i look so good that a worker was hitti...,damn ik i look so good that a worker was hitti...,2025-06-10 19:44:17+00:00


In [20]:
df.shape
df = df.drop(columns=['unnamed: 0', 'created_at'], axis=0)


In [21]:
df

,post_text,clean_text
0,It's just over 2 years since I was diagnosed w...,it's just over 2 years since i was diagnosed w...
1,"It's Sunday, I need a break, so I'm planning t...","it's sunday, i need a break, so i'm planning t..."
2,Awake but tired. I need to sleep but my brain ...,awake but tired. i need to sleep but my brain ...
3,RT @SewHQ: #Retro bears make perfect gifts and...,rt : #retro bears make perfect gifts and are g...
4,It’s hard to say whether packing lists are mak...,it’s hard to say whether packing lists are mak...
...,...,...
20095,@AbnRgr504 @LangmanVince That makes sense...sa...,that makes sense...sad...they should be the on...
20096,@stacy_muur @opensea @monad_xyz @aztecnetwork ...,alpha will fucked us
20097,@shellshockkk That’s some fucked up shit right...,that’s some fucked up shit right there. how mu...
20098,damn ik i look so good that a worker was hitti...,damn ik i look so good that a worker was hitti...


In [43]:
from transformers import pipeline

classifier = pipeline("text-classification", 
                      model="bhadresh-savani/distilbert-base-uncased-emotion", 
                      top_k=1)  # model will return multiple labels


d:\mental_health_X\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--bhadresh-savani--distilbert-base-uncased-emotion. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to r

In [44]:
def classify_emotion_bert(text):
    try:
        result = classifier(str(text)[:512])[0]  # Truncate and classify
        label = result['label'].lower()
        depressed_labels = ['sadness', 'fear', 'anger', 'disgust']
        return 'depressed' if label in depressed_labels else 'undepressed'
    except:
        return 'undepressed'

In [45]:
df['mental_health'] = df['clean_text'].apply(classify_emotion_bert)

In [50]:
df

,post_text,clean_text,sentiment,mental_health
0,It's just over 2 years since I was diagnosed w...,it's just over 2 years since i was diagnosed w...,negative,undepressed
1,"It's Sunday, I need a break, so I'm planning t...","it's sunday, i need a break, so i'm planning t...",neutral,undepressed
2,Awake but tired. I need to sleep but my brain ...,awake but tired. i need to sleep but my brain ...,negative,undepressed
3,RT @SewHQ: #Retro bears make perfect gifts and...,rt : #retro bears make perfect gifts and are g...,positive,undepressed
4,It’s hard to say whether packing lists are mak...,it’s hard to say whether packing lists are mak...,positive,undepressed
...,...,...,...,...
20095,@AbnRgr504 @LangmanVince That makes sense...sa...,that makes sense...sad...they should be the on...,positive,undepressed
20096,@stacy_muur @opensea @monad_xyz @aztecnetwork ...,alpha will fucked us,negative,undepressed
20097,@shellshockkk That’s some fucked up shit right...,that’s some fucked up shit right there. how mu...,negative,undepressed
20098,damn ik i look so good that a worker was hitti...,damn ik i look so good that a worker was hitti...,negative,undepressed


In [22]:
analyzer = SentimentIntensityAnalyzer()

In [23]:
def get_sentiment(text):
  score = analyzer.polarity_scores(str(text))
  if score['compound']>=0.05:
    return 'positive'
  elif score['compound'] <= -0.05:
        return 'negative'
  else:
        return 'neutral'

In [36]:
def classify_depression(text):
    keywords = ['depressed', 'hopeless', 'suicidal','sad',
                'anxious', 'lonely', 'crying', 'kill myself']
    return 'depressed' if any(word in text.lower() for word in keywords) else 'undepressed'

In [37]:
df['sentiment'] = df['clean_text'].apply(get_sentiment)

In [38]:
df['mental_health'] = df['clean_text'].apply(classify_depression)

In [39]:
df

,post_text,clean_text,sentiment,mental_health
0,It's just over 2 years since I was diagnosed w...,it's just over 2 years since i was diagnosed w...,negative,undepressed
1,"It's Sunday, I need a break, so I'm planning t...","it's sunday, i need a break, so i'm planning t...",neutral,undepressed
2,Awake but tired. I need to sleep but my brain ...,awake but tired. i need to sleep but my brain ...,negative,undepressed
3,RT @SewHQ: #Retro bears make perfect gifts and...,rt : #retro bears make perfect gifts and are g...,positive,undepressed
4,It’s hard to say whether packing lists are mak...,it’s hard to say whether packing lists are mak...,positive,undepressed
...,...,...,...,...
20095,@AbnRgr504 @LangmanVince That makes sense...sa...,that makes sense...sad...they should be the on...,positive,depressed
20096,@stacy_muur @opensea @monad_xyz @aztecnetwork ...,alpha will fucked us,negative,undepressed
20097,@shellshockkk That’s some fucked up shit right...,that’s some fucked up shit right there. how mu...,negative,undepressed
20098,damn ik i look so good that a worker was hitti...,damn ik i look so good that a worker was hitti...,negative,undepressed


In [40]:
df['mental_health'].value_counts()

mental_health
undepressed    19840
depressed        260
Name: count, dtype: int64

In [41]:
depressed_df = df[df['mental_health']=='depressed']
depressed_df

,post_text,clean_text,sentiment,mental_health
52,The pain and heartbreak of tragedy is such a c...,the pain and heartbreak of tragedy is such a c...,negative,depressed
62,@yorkshireguy @Sarah_Booker I can't shake the ...,i can't shake the image of the unsuspecting pe...,negative,depressed
68,RT @BBCSussex: This very sad footage has been ...,rt : this very sad footage has been released o...,negative,depressed
89,Feeling a strange mix of excited and sad. Such...,feeling a strange mix of excited and sad. such...,negative,depressed
139,"This I know: don't crochet when tipsy, angry o...","this i know: don't crochet when tipsy, angry o...",negative,depressed
...,...,...,...,...
20090,It’s sad that we don’t flex with the cameras a...,it’s sad that we don’t flex with the cameras a...,negative,depressed
20091,@PrimeTime_Vol You are right. But It is sad th...,you are right. but it is sad though. she might...,positive,depressed
20093,@liedetectors42 So sad.,so sad.,negative,depressed
20094,@abdool_moh Exactly \nIt’s Nigeria that is los...,exactly \nit’s nigeria that is losing \nwe kee...,positive,depressed
